# Simulating moving objects

## Contents
+ Simulating realistic motion using Newton's laws of motion
+ Introducing the velocity and acceleration vectors
+ Using Euler's method to approximate the position of a moving objects
+ Finding the exact trajectory of a moving object using calculus

## Introduction

To implement motion in a videogame, like our *Asteroid* bootleg, we need to use many of the calculus concepts from the previous chapter.

The numerical entities we will consider are the $ x $ and $ y $ positions of the asteroids and the spaceship. Those will be functions of the time, so we will have $ x(t) $ and $ y(t) $.

We will consider the following physical functions:
+ *velocity* &mdash; the derivative of the position function with respect to time.
+ *acceleration* &mdash; the derivative of velocity with respect to time.

As we have two functions $ x(t) $ and $ y(t) $ to determine the position of objects, we'll have two velocity functions and two acceleration functions, which will allow us to think in terms of vectors as well.

Our first goal will consist in get the asteroids moving. To do that, we'll provide them with random, constant velocity functions. Then, we'll *integrate* these functions in real time to get their positions. We will use *Euler's method* to do that, which is a mathematically similar method for integration, with the advantage that it can be done as the game runs.

The second goal will be to let the user move the spaceship. When the user presses any of the direction keys, the spaceship should *accelerate* on the direction of the pressed key, which means that the velocity will change, and as a result the position of the spaceship as well.

Our third and final goal will consist in deep diving on the *Euler's method* as a way to obtain an approximation of the integral, and compare it with the exact solutions.

## Simulating a constant velocity motion

In everyday usage, velocity is synonym with speed. In math and physics, velocity includes both the notions of speed and direction of motion &mdash; this means velocity is a vector!

In our game, we will enhance the asteroid objects with a random velocity vector $ (vx, vy) $, and consider those to be the constant values of the derivatives of the position functions with respect to time.

$
x'(t) = vx \\
y'(t) = vy
$

For simplicity, we will talk about:

$
s(t) = (x(t), y(t)) \\
v(t) = (x'(t), y'(t))
$

As if they were vector-valued functions, but it just means that those are individual functions that take time and return values that we will interpret as vectors.

### Adding velocities to the asteroids

Let's recap where we left our *Asteroids* bootleg. We created a hierarchy of classes that modeled our game entities first in a *math* world of coordinates and vectors and some functions that translated that model into the *game* world of pixels and frames.

In the last step of the game, we even made our spacechip to rotate as can be seen when running [Asteroids: step 3](../e00-my-lib/asteroids_step3.py). In that stage, all the asteroids were static.

In [4]:
from asteroids_step3 import main 

main()

pygame 2.0.0 (SDL 2.0.12, python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 0

This *math* model was supported by the `PolygonModel` class. As all of our game entities inherit from it, and all of them are subject of being moved, we will start by adding the velocity vector components as properties:

```python
class PolygonModel():
    def __init__(self, points):
        self.points = points
        self.rotation_angle = 0
        # x, y keeps track of the center of the polygon
        self.x = 0
        self.y = 0

        # vx, vy are the velocity components
        self.vx = 0
        self.vy = 0
```

Next, to make our asteroids to move erratically we can assign them with random velocity values:

```python
class Asteroid(PolygonModel):
    def __init__(self):
        sides = randint(5, 9)
        points = [to_cartesian((uniform(0.5, 1.0), 2 * pi * i / sides))
            for i in range(0, sides)
        ]
        super().__init__(points)

        # assign random velocity values
        self.vx = uniform(-1, 1)
        self.vy = uniform(-1, 1)
```

That will make `vx` and `vy` to take a random value between $ (-1, 1) $, with negative values for the velocity meaning the position of the corresponding coordinate will decrease, instead of increase.

### Updating the game engine to move the asteroids

With the velocity values given, we need to use them to update the the game entities position `x` and `y`.

If $ \Delta t $ is the elapsed time between frames, we know that x will change according to:

$ 
x = x + vx \times \Delta t \\
y = y + vy \times \Delta t
$

This is a good way to do it because velocities are constant, and we know that $ v = \frac{s}{t} $.

Thus, the way to make the asteroids move will be to update our `PolygonModel` to include a `move()` method that will update the game entity position:

```python
class PolygonModel():
...
    def move(self, milliseconds):
        dx, dy = (self.vx * milliseconds / 1000, self.vy * milliseconds / 1000)
        self.x, self.y = add((self.x, self.y), (dx, dy))
```

This has been the first application of algorithm known as the *Euler's method*.

The algorithm consists of keeping track of the value of one or more functions (in our case, $ x(t) $, $ y(t) $) and their derivatives ($ x'(t) $ and $ y'(t) $) and updating the functions according to their derivatives in each step.

This will be exact if the derivatives are constant, but it will even be a good approximation if they're not.

### Keeping the asteroids on the screen

As our game model space (that is the screen space) is fairly limited, any asteroid with a random velocity will be bound to drift off the screen at some point in time.

To keep the asteroids within the screen are, we can add some extra logic to keep the $ x $ and $ y $ coordinates between the min and max *math* values of $ (-10, 10) $.

One common solution in videogame scenarios (and the one that *Asteroids* used), consists in subtracting the width (or height) of the screen when one of the coordinate went over the limit.

This effect can be seen in the picture below:

![Teleportation](../images/teleportation.png)

This can be written in the `move()` method:

```python
class PolygonModel():
...
    def move(self, milliseconds):
        dx, dy = (self.vx * milliseconds / 1000, self.vy * milliseconds / 1000)
        self.x, self.y = add((self.x, self.y), (dx, dy))

        if self.x < -10:
            self.x += 20
        if self.y < -10:
            self.y += 20
        if self.x > 10:
            self.x -= 20
        if self.y > 10:
            self.y -= 20
```

With that we're done with the *math* model of our game, and we just need to adjust the *game* model so that we add the call to move the asteroid in every game loop iteration.

```python
def main():
...
    while not done:
        clock.tick()
...
        milliseconds = clock.get_time()
...
        for asteroid in asteroids:
            asteroid.move(milliseconds)

...
```

Those unremarkable 2 lines make all of our asteroids move according to the given velocity.

In [4]:
from asteroids_step4 import main

main()

SystemExit: 0

## Simulating acceleration